# Identifying the topics based on questions asked by Amazon customers

### What is topic modelling?
It is an unsupervised approach used for finding and observing the bunch of words (called Topics) in large clusters of texts.

### Algorithm used:
There are many algorithms available for Topic Modelling such as LSA (Latent Semantic Analysis), NMF (Non-Matrix Factorization) and LDA (Latent Dirichlet Algorithm). I'm using LDA as it provides more accurate results and scales well for large text corpuses.

### A brief intro to LDA algorithm:
LDA is a matrix factorization technique which assumes sentences are made up of mixture of words and tries to figure out what words would create those sentences in the first place. It is based on probabilistic graphical modelling. The algorithm works as follows:<br>
1. In the initialization stage, each word is assigned to a random topic.
2. Iteratively, the algorithm goes through each word and reassigns the word to a topic taking into consideration:<br>
    What’s the probability of the word belonging to a topic?<br>
    What’s the probability of the document to be generated by a topic?
[Source]('https://nlpforhackers.io/topic-modeling/')

### Data Source:
http://jmcauley.ucsd.edu/data/amazon/qa/. Click on the Automotive category file to download the json file having the customer queries.

#### Step 1: Import the required libraries

In [18]:
#Hold the data in a table
import pandas as pd

#Read the zip files
import gzip

#Ignore warnings
import warnings
warnings.filterwarnings("ignore")

#Data Preprocessing
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

#For visualizing the LDA topics
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

#### Step 2: Import the data into dataframe

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

data = getDF('qa_Automotive.json.gz')

#### Step 3: Extract the text column and assign it to an index

In [3]:
def prepare_data(file_name):
    data_text = data[['question']]
    data_text['index'] = data_text.index
    return data_text
documents=prepare_data(data)

Overview of the file

In [4]:
documents.head()

,question,index
0,What is the most useful length to get?,0
1,Are these cables made of copper or aluminum?,1
2,I bought the Red Extra Heavy Duty. Is that too...,2
3,"Hi, Being 20ft 4gauge how heavy is this?",3
4,Do these cables come with a bag?,4


#### Step 4: Reducing the words to its common  base form
Now, we need to reduce the words which are in its plural or derivational forms into its common base form. This will help us collect the similar words from all the corpuses. Now the question is how do we do that? Here, stemming and lemmatization comes to the rescue.<br><br>
**What is Stemming?** <br>
The process of chopping off the ends of words, often includes the removal of derivational fixes. E.g. cats -> cat<br>
There are two types of stemmer: SnowballStemmer and PorterStemmer. We will be using SnowballStemmer as it gives a more meaningful result compared to PorterStemmer().<br>

**What is Lemmatization?** <br>
The process of removal of inflectional endings only and returning the base or dictionary form of a word, which is known as lemma. E.g. running -> run.<br>

In [5]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
stemmer = SnowballStemmer('english')

Next, we need to remove the stopwords and words whose length is less than or equal to three as they won't add much value to our model. We will implement it using gensim library as it already has a tokenization function and built-in library of stopwords.

In [6]:
def remove_stopwords_and_shortwords(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

Let's look at an example of how it works:

In [7]:
sample = documents[documents['index'] == 25].values[0][0]

print('original document: ')
words = []
for word in sample.split(' '):
    words.append(word)
print(words)
print('\n\n Tokenized and lemmatized document: ')
print(remove_stopwords_and_shortwords(sample))

original document: 
['It', 'does', 'not', 'state', 'what', 'RPM', 'this', 'motor', 'is,', 'does', 'anyone', 'know?']


 Tokenized and lemmatized document: 
['state', 'motor', 'know']


Now, we will apply the same on our dataset.

In [8]:
processed_docs = documents['question'].map(remove_stopwords_and_shortwords)

Overview of the processed docs

In [9]:
processed_docs[:10]

0                      [use, length]
1           [cabl, copper, aluminum]
2    [buy, extra, heavi, duti, size]
3                      [gaug, heavi]
4                       [cabl, come]
5        [wire, pair, surpris, wire]
6                       [amp, handl]
7              [cabl, boost, school]
8                      [use, length]
9           [cabl, copper, aluminum]
Name: question, dtype: object

#### Step 5: Build a dictionary of words present in the dataset

In [10]:
dictionary = gensim.corpora.Dictionary(processed_docs)
print('There are total '+ str(len(dictionary))+' words in the dictionary.')

There are total 14469 words in the dictionary.


An overview of the dictionary:

In [11]:
count = 0
for index, word in dictionary.iteritems():
    print(index, word)
    count += 1
    if count > 10:
        break

0 length
1 use
2 aluminum
3 cabl
4 copper
5 buy
6 duti
7 extra
8 heavi
9 size
10 gaug


#### Step 6: Filter the dictionary based on frequency of words. 
Here, I have ignored the words which have appeared in less than 4 queries and the ones who are present in at most 50% of the documents.

In [12]:
dictionary.filter_extremes(no_below=4, no_above=0.5)
print('No. of words in the dictionary after filtering: '+ str(len(dictionary)))

No. of words in the dictionary after filtering: 4705


#### Step 7: Create a list having the index value and the frequency of the word for every question in the dataset.

In [13]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[:5]

[[(0, 1), (1, 1)],
 [(2, 1), (3, 1), (4, 1)],
 [(5, 1), (6, 1), (7, 1), (8, 1), (9, 1)],
 [(8, 1), (10, 1)],
 [(3, 1), (11, 1)]]

A sample:

In [14]:
bow_doc_sample = bow_corpus[0]

for i in range(len(bow_doc_sample)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_sample[i][0], 
                                                     dictionary[bow_doc_sample[i][0]], 
                                                     bow_doc_sample[i][1]))

Word 0 ("length") appears 1 time.
Word 1 ("use") appears 1 time.


#### Step 8: Implement the LDA model

In [15]:
lda_model = gensim.models.ldamodel.LdaModel(bow_corpus, num_topics=5, id2word=dictionary, passes=5,minimum_probability=0.01, random_state=20)

In [16]:
for idx, topic in lda_model.print_topics(-1,num_words=7):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.041*"light" + 0.036*"work" + 0.036*"bulb" + 0.032*"toyota" + 0.019*"dodg" + 0.018*"inch" + 0.015*"dimens"
Topic: 1 
Words: 0.036*"instal" + 0.031*"come" + 0.025*"mount" + 0.025*"need" + 0.024*"wire" + 0.018*"light" + 0.015*"work"
Topic: 2 
Words: 0.031*"size" + 0.031*"tire" + 0.024*"need" + 0.018*"color" + 0.018*"order" + 0.017*"black" + 0.016*"know"
Topic: 3 
Words: 0.091*"work" + 0.036*"seat" + 0.033*"model" + 0.025*"honda" + 0.021*"ford" + 0.020*"chevi" + 0.019*"cover"
Topic: 4 
Words: 0.030*"batteri" + 0.027*"say" + 0.019*"unit" + 0.017*"plug" + 0.015*"light" + 0.015*"power" + 0.015*"charg"


#### Step 9: Visualize the topics
I have used pyLDAvis library which is designed to help users interpret the topics in a topic model that has been fit to a corpus of text data. The package extracts information from a fitted LDA topic model to inform an interactive web-based visualization.[Source]('https://github.com/bmabey/pyLDAvis')

In [20]:
vis1=pyLDAvis.gensim.prepare(lda_model,bow_corpus,dictionary, R=7)
vis1

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.115764  0.174179       1        1  24.264500
4     -0.222747  0.007072       2        1  20.574907
2     -0.134129 -0.264317       3        1  18.964205
3      0.322417 -0.152108       4        1  18.642494
0      0.150223  0.235174       5        1  17.553894, topic_info=     Category          Freq      Term         Total  loglift  logprob
term                                                                 
79    Default  10520.000000      work  10520.000000   7.0000   7.0000
148   Default   3454.000000    instal   3454.000000   6.0000   6.0000
674   Default   2600.000000      seat   2600.000000   5.0000   5.0000
1464  Default   2454.000000      bulb   2454.000000   4.0000   4.0000
26    Default   2381.000000     model   2381.000000   3.0000   3.0000
9     Default   2326.000000      size   2326.000000   2.0000   2.0000
344   Default   2212.000000    toyota   2212.000000   1.0000   1.0000
148    Topic1   3453.489014    instal   3454.304932   1.4159  -3.3162
150    Topic1   2365.153809     mount   2365.968750   1.4158  -3.6948
14     Topic1   2310.898438      wire   2311.711914   1.4158  -3.7180
81     Topic1   1037.681274    switch   1038.498047   1.4154  -4.5186
503    Topic1    967.555359     remov    968.388611   1.4153  -4.5886
256    Topic1    841.779358   bracket    842.593018   1.4152  -4.7278
221    Topic1    810.175171     remot    810.988708   1.4152  -4.7661
11     Topic1   2976.346191      come   4805.895020   0.9370  -3.4649
166    Topic1   1266.130005    includ   1649.621216   1.1516  -4.3196
198    Topic1   2341.288086      need   5533.953125   0.5560  -3.7049
84     Topic1   1668.351440     light   5751.640625   0.1785  -4.0438
79     Topic1   1385.436035      work  10520.366211  -0.6111  -4.2296
44     Topic2   2383.276855   batteri   2384.081055   1.5808  -3.5222
36     Topic2   2166.911133       say   2167.716797   1.5807  -3.6174
157    Topic2   1386.432373      plug   1387.240234   1.5805  -4.0639
49     Topic2   1173.170288     charg   1173.974243   1.5804  -4.2310
309    Topic2    966.325500    accord    967.132385   1.5803  -4.4249
386    Topic2    763.539612     price    764.342285   1.5800  -4.6605
156    Topic2    742.393433     water    743.199524   1.5800  -4.6885
154    Topic2   1560.688965      unit   1564.046387   1.5789  -3.9455
39     Topic2   1229.338745     power   1558.274902   1.3440  -4.1842
84     Topic2   1236.371094     light   5751.640625   0.0438  -4.1785
9      Topic3   2325.226074      size   2326.017578   1.6623  -3.4653
339    Topic3   2313.531738      tire   2314.324707   1.6623  -3.4704
264    Topic3   1371.092041     color   1371.883301   1.6620  -3.9935
384    Topic3   1301.977783     order   1302.773560   1.6620  -4.0453
218    Topic3   1265.787354     black   1266.580811   1.6620  -4.0735
52     Topic3    993.316528   purchas    994.113892   1.6618  -4.3159
340    Topic3    946.429749  wrangler    947.219299   1.6618  -4.3642
198    Topic3   1773.490112      need   5533.953125   0.5247  -3.7362
258    Topic3   1112.983032      door   1576.549805   1.3144  -4.2021
21     Topic3   1172.557983      know   2585.602295   0.8718  -4.1500
674    Topic4   2599.744141      seat   2600.552002   1.6794  -3.3366
26     Topic4   2380.328369     model   2381.138184   1.6794  -3.4248
743    Topic4   1488.362793     chevi   1489.168335   1.6792  -3.8944
467    Topic4   1140.448120    nissan   1141.254272   1.6790  -4.1606
757    Topic4    989.163208    mirror    989.970459   1.6789  -4.3029
1341   Topic4    953.573364       mat    954.381104   1.6789  -4.3396
213    Topic4    893.402588     engin    894.210205   1.6788  -4.4048
79     Topic4   6674.413574      work  10520.366211   1.2247  -2.3938
310    Topic4   1851.840210     honda   2645.082275   1.3232  -3.6759
301    Topic4   1501.231934      ford   1916.277710   1.4356  -3.8858
493    To

#### Step 10: Intepretation of topics obtained:

Now, let's try to identify the category of questions asked from the words provided for each topic:
1. Topic 1 (install, come, mount, wire, need, light, work) - **Installation**
2. Topic 2 (battery, say, unit, plug, light, power, charge) - **Battery & Charging**
3. Topic 3 (size, tire, need, color, order, black, know) - **Tire**
4. Topic 4 (work, seat, honda, model, chevi, ford, cover) - **Model specific**
5. Topic 5 (light, bulb, work, toyota, dodge, inch, dimension) - **Cannot deduce a valid topic** <br>

There are cases when the some of the topics obtained don't make much sense. One way to fix it would be to play around the hyperparameters in order to improve the results or sometimes its okay to just ignore them. Here, we can see that words in Topic 5 are not making much sense for categorizing it into specific category.